# Real-Time TDDFT
*Roberto Di Remigio*

## Running the simulations

You will, of course, need the appropriate version of ReSpect. The following are sample files for the water molecule using HF and uncontracted cc-pVDZ basis set. Change as needed.

#### SCF step

```
scf:

     geometry:
               O  0.000000  0.000000  0.000000
               H  0.000000  0.000000  0.940000
               H  0.903870  0.000000 -0.258105

     method:         hf
     initmo:         atomic 
     nc-model:       point
     charge:         0
     multiplicity:   1
     maxiterations:  30
     convergence:    1.e-7 

     basis:
            H: ucc-pvdz
            O: ucc-pvdz
```

Assuming the input file is named `scf.inp`, run with:

```bash
respect --scf --inp=scf
```

the `scf.out_scf` output file and `scf.50` checkpoint file will be generated. If the calculation ends successfully, that is.


#### TDSCF step(s)
The TDSCF trajectory will, most likely, be calculated in batches of $n_\mathrm{steps}$ points. The first batch of points will restart from the SCF step, perform $n_\mathrm{steps}$, and save the results to the corresponding `.out_tdscf` output and `.50` checkpoint file.
The template TDSCF step input is:

```
tdscf:
       spectroscopy:    eas
       solver:          magnus
       time-steps:      nsteps x 0.005
       maxiterations:   6 
       convergence:     1.0e-7
       checkpoint:      3
       field:           
             model:     delta
             amplitude: 0.2
             direction: 1.0 0.0 0.0
```

this one is for the perturbation applied in the $x$ direction. We will thus have three templates, one per direction, named `tdscf_x.inp`, `tdscf_y.inp`, and `tdscf_z.inp`, respectively.

Before starting the simulation, we need to substitute `nsteps` with the number of actual steps to be run in the calculation. We'll just look at the $x$ direction, but the gist should be evident. To keep everything in order:

```bash
nsteps_previous=0
nsteps_next=5
cp tdscf_x.inp ${nsteps_next}_tdscf_x.inp
sed -i -e "s/nsteps/${nsteps_next}/g" ${nsteps_next}_tdscf_x.inp
```

The launcher script will look for a similarly named checkpoint file to start the calculation from. Instead of copying the one from SCF (or a previous TDSCF batch of points) we use symlinks:

```bash
if [ "$nsteps_previous" -eq "0" ]; then
  ln -sf scf.50 ${nsteps_next}_tdscf_x.50
else
  ln -sf ${nsteps_previous}_tdscf_x.50 ${nsteps_next}_tdscf_x.50
fi 
```

We are finally ready to roll with:

```bash
if [ "$nsteps_previous" -eq "0" ]; then
  respect --tdscf --inp=${nsteps_next}_tdscf_x 
else
  respect --tdscf --inp=${nsteps_next}_tdscf_x --restart
fi 
```

## Obtain raw data from simulation output

Once we have the output files, we are ready to extract the data from it. We use the `spectrum.py` script. I have a slightly modified version that:
1. Reads in data using a regex
2. Outputs the discrete Fourier transform to NumPy arrays, rather than to plain-text